# Exploratory Data Analysis

The data set is quite strange in how it is tabulated. Each column is a year. Every day a new value for the previous day is added but the earliest value is deleted meaning there is a limit to how many values the TSA publishes to its website simultaneously.

The data is tabulated so that one can compare the number of flights on the same date of the previous year. As such, it is not arranged chronologically. We will have to mess with some date reformatting to get the data to it's desired format.

In [143]:
import pandas as pd

In [144]:
df = pd.read_csv("Data1.csv")

## Formatting and Cleaning Data

Our first goal is to change the data into a time series format, we have to build date-sequences, and then overlay them onto the correct years. 

If you examine the data from the TSA website, you will notice that each year's column begins in the previous year and ends in the current year. Therefore, our data begins from 2018.

In [145]:
df.head()

,Unnamed: 0,Date,2023,2022,2021,2020,2019
0,0,4/10/2023,"2,511,861","2,188,196","1,468,972","102,184","2,484,580"
1,1,4/9/2023,"2,375,705","2,300,909","1,561,495","90,510","2,446,801"
2,2,4/8/2023,"2,093,075","1,959,523","1,378,237","93,645","2,059,142"
3,3,4/7/2023,"2,475,368","2,327,849","1,549,181","108,977","2,590,499"
4,4,4/6/2023,"2,508,487","2,227,151","1,510,829","104,090","2,487,398"


First we create our own date ranges labeled with the correct year since the date column in the original data only contains 2022-2023 dates. We use pandas date ranges in order to do this.

The original data does not contain the leap-day (2020 was a leap year) so we have to remove this day from our 2020 date range.

In [146]:
pd.to_datetime(df["Date"].iloc[0])

Timestamp('2023-04-10 00:00:00')

In [185]:
#Converts a date in the form of a string to 
#date: date in form of string with mm/dd/yyyy
#year: in form of string to a timestamp with the desired year
def format_date(date, year):
    date_list = date.split("/")
    date_list[2] = year
    date = "/".join(date_list)
    date = pd.to_datetime(date)
    return date

In [186]:
format_date

<function __main__.format_date(date, year)>

In [198]:
def format_to_ts(df, years, truncated_year):
    ts_df = pd.DataFrame({"Date": [], "Checkins": []})
    for year in years:
        #Use our formatting function in a lambda so we can map it on the dataframe
        x = lambda date: format_date(date, year)
        curr_df = df[["Date",year]]
        curr_df["Date"] = curr_df["Date"].map(x)
        curr_df.rename(columns = {year: "Checkins"}, inplace = True)
        ts_df = ts_df.append(curr_df)
    #drops the nan values
    x = lambda date: format_date(date, truncated_year)
    truncated_df = df[["Date", truncated_year]].dropna(inplace = True)
    truncated_df = truncated_df.map(x)
    truncated_df.rename(columns = {truncated_year : "Checkins"}, inplace = True)
    ts_df = ts_df.append(truncated_df)
    return ts_df

In [199]:
years = ["2019", "2020", "2021", "2022"]
truncated_year = "2023"

In [200]:
ts_df = format_to_ts(df, years, truncated_year)

<ipython-input-198-98b84eabf003>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_df["Date"] = curr_df["Date"].map(x)
<ipython-input-198-98b84eabf003>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  curr_df.rename(columns = {year: "Checkins"}, inplace = True)
<ipython-input-198-98b84eabf003>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ts_df = ts_df.append(curr_df)
<ipython-input-198-98b84eabf003>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

AttributeError: 'NoneType' object has no attribute 'map'

In [201]:
ts_df.sort_values(by = "Date")

,Date,Checkins
99,2019-01-01,"2,202,111"
98,2019-01-02,"2,150,571"
97,2019-01-03,"1,975,947"
96,2019-01-04,"2,229,391"
95,2019-01-05,"2,044,043"
...,...,...
104,2022-12-27,"2,163,901"
103,2022-12-28,"2,143,566"
102,2022-12-29,"2,168,039"
101,2022-12-30,"2,367,709"


Now we concatenate our date ranges in chronological order as well as the checkin values. We then form a data-frame which we will later use in time series analysis.

In [108]:
vals1 = df["2019"][::-1]
vals2 = df["2020"][::-1]
vals3 = df["2021"][::-1]
vals4 = df["2022"][::-1]
vals5 = df["2023"][::-1]

In [112]:
vals5.dropna(inplace = True)

In [114]:
range5

DatetimeIndex(['2022-04-12', '2022-04-13', '2022-04-14', '2022-04-15',
               '2022-04-16', '2022-04-17', '2022-04-18', '2022-04-19',
               '2022-04-20', '2022-04-21',
               ...
               '2022-12-22', '2022-12-23', '2022-12-24', '2022-12-25',
               '2022-12-26', '2022-12-27', '2022-12-28', '2022-12-29',
               '2022-12-30', '2022-12-31'],
              dtype='datetime64[ns]', length=264, freq='D')

In [96]:
df["2019"][::-1]

363    2,208,688
362    2,317,381
361    2,616,158
360    2,457,133
359    1,988,205
         ...    
4      2,487,398
3      2,590,499
2      2,059,142
1      2,446,801
0      2,484,580
Name: 2019, Length: 364, dtype: object

In [ ]:
def to_int(s):
    if s == 'nan':
        return 'nan'
    s = str(s)
    return int(s.replace(",",""))

data.dropna(inplace = True)
data.Checkins = data.Checkins.map(to_int)